In [58]:
import tensorflow as tf
import random
import numpy as np
from vizdoom import *
from collections import deque
from skimage import transform

C:\Users\vedan\AppData\Local\conda\conda\envs\dlwin36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Setup environment 

In [11]:
def create_env():
    game = DoomGame()
    game.load_config("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.cfg")
    game.set_doom_scenario_path("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.wad")
#     game.init()
    
    shoot = [0,0,1]
    left = [1,0,0]
    right = [0,1,0]
    possible_action = [shoot,left,right]
    
    return game,possible

In [9]:
def test():
    game = DoomGame()
    game.load_config("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.cfg")
    game.set_doom_scenario_path("./ViZDoom-1.1.5pre-Win-Python36-x86_64/ViZDoom-1.1.5pre-Win-Python36-x86_64/vizdoom/scenarios/basic.wad")
#     game.set_mode(Mode.ASYNC_PLAYER)

    game.init()

    shoot = [0,0,1]
    left = [1,0,0]
    right = [0,1,0]

    episodes = 10
    for i in range(episodes):
        game.new_episode()
        while not game.is_episode_finished():
            state = game.get_state()
            img = state.screen_buffer
            misc = state.game_variables
            action = random.choice([shoot, left, right]) # A random agent
            reward = game.make_action(action)
            time.sleep(0.02) # Just to make it slower
            
            
        print("Total reward: " + str(game.get_total_reward()))
    game.close()

In [57]:
class Inp_data:

    def __init__(self):
        self.que = deque(maxlen=4)
    
    def add(self,frame):
        self.que.append(frame)
        return np.stack(self.que,axis=0)

In [56]:
def preprocess(frame):
    
    frame = frame.mean(axis=0,dtype = np.int)
    crop_frame = frame[30:-10,30:-30]
    frame = crop_frame/255
    return transform.resize(frame,[84,84])

In [59]:
imgsize = [84,84,4]

In [ ]:
class Dqn:
    
    def __init__(self):
        with tf.variable_scope("dqn_var")
            self._input = tf.placeholder(dtype=tf.float32,shape=[None,imgsize[0],imgsize[1],imgsize[2]],name="input")
            self._output = tf.placeholder(dtype=tf.float32,shape=[None,3],name="output")
            
            self._conv1 = tf.layers.conv2d(self._input,
                                     filters=32,
                                     kernel_size=[8,8],
                                     strides=[4,4],
                                     padding="VALID"
                                     kernel_initializer=tf.initializers.contrib.layers.xavier_initializer_conv2d, 
                                     name="conv1")
            
            self._conv1_batch = tf.layers.batch_normalization(self._conv1,trainable=True,name="b_conv1")
            self._conv1_out = tf.nn.elu(self._conv1_batch,name = "e_conv1")
            
            self._conv2 = tf.layers.conv2d(self._conv1_out,
                                     filters=64,
                                     kernel_size=[4,4],
                                     strides=[2,2],
                                     padding="VALID"
                                     kernel_initializer=tf.initializers.contrib.layers.xavier_initializer_conv2d, 
                                     name="conv2")
            
            self._conv2_batch = tf.layers.batch_normalization(self._conv2,trainable=True,name="b_conv2")
            self._conv2_out = tf.nn.elu(self._conv2_batch,name = "e_conv2")
            
            self._conv3 = tf.layers.conv2d(self._conv2_out,
                                     filters=128,
                                     kernel_size=[4,4],
                                     strides=[2,2],
                                     padding="VALID"
                                     kernel_initializer=tf.initializers.contrib.layers.xavier_initializer_conv2d, 
                                     name="conv3")
            
            self._conv3_batch = tf.layers.batch_normalization(self._conv3,trainable=True,name="b_conv3")
            self._conv3_out = tf.nn.elu(self._conv3_batch,name = "e_conv3")
            
            
            self._flat = tf.layers.flatten(self._conv3_out,name="flat")
            
            self._dense1 = tf.layers.dense(self._flat,
                            units=512,
                            activation=tf.nn.elu,
                            kernel_initializer=tf.initializers.contrib.layers.xavier_initializer_conv2d)
             
            self._out = tf.layers.dense(self._dense1,
                            units=512,
                            kernel_initializer=tf.initializers.contrib.layers.xavier_initializer_conv2d)
            
            
            